# UppASD bcc Fe: Single Simulation Tutorial

This notebook is a **step-by-step guide** for running one atomistic spin-dynamics simulation of bcc Fe with UppASD.

It is designed for students and researchers who want to understand the full workflow from model construction to time-series analysis of magnetization and energy.

## Scientific objective

We run a single finite-temperature simulation and inspect how the system evolves over iterations by analyzing:
- average magnetization components and magnitude
- total energy relaxation

## Learning goals
- Define a minimal bcc lattice and magnetic state
- Add shell-resolved exchange interactions
- Configure initial phase + simulation controls
- Run UppASD and parse outputs with `ASDResults`
- Build and interpret magnetization/energy time-series plots

## Recommended execution order
Run cells from top to bottom without skipping. If you change setup parameters, rerun from the modified cell onward.

### Note:
This tutorial uses the python based interface to define the exchange interactions. For daily/production use, the file-based approach using `jfile`, `posfile`, and `momfile` is recommended. Their usage is documented in the manual and in other tutorials.  
The notebook/python wrappers always creates the input files as an intermediate step and they can be found in the run directory.

## Units and conventions used in this tutorial

Unless explicitly stated otherwise, this notebook uses UppASD default output conventions:
- **Time**: seconds (`s`)
- **Temperature**: kelvin (`K`)
- **Energy**: milli-Rydberg (`mRy`)
- **Magnetization / moments**: Bohr magneton (`μ_B`)
- **Exchange parameters (`Jij`)**: `mRy`
- **DMI parameters (`Dij`)**: `mRy`
- **Binder cumulant**: dimensionless

For this single-run notebook, we primarily analyze magnetization (`μ_B`) and total energy (`mRy`) as functions of simulation time.

In [ ]:
import importlib.util
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Install UppASD if missing in current kernel environment
if importlib.util.find_spec("uppasd") is None:
    print("Installing uppasd…")
    !pip install --pre --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple uppasd

# Core UppASD classes for model setup, execution, and results parsing
from uppasd.core.system import SpinSystem
from uppasd.core.exchange import ExchangeShellTable, DMIShellTable
from uppasd.input.inputdata import ASDInput
from uppasd.run.simulator import ASDWorkspace, UppASDSimulator
from uppasd.core.results import ASDResults

# Temperature-token helpers (useful for switching initial-phase syntax)
from uppasd import set_temperature_token, validate_temperature_token

print("✅ UppASD imported successfully")

## Step 1 — Define the crystal and initial magnetic state

We start from a minimal bcc model to keep the physics and code path transparent.

### What each object means
- `cell`: lattice vectors that define periodic geometry
- `positions`: atom coordinates in the unit cell
- `species`: integer species labels (important for multi-component systems)
- `moments`: initial spin vectors per atom

Here we initialize moments along `+z`, which gives a simple reference state for observing relaxation behavior.

### Note: 
Using this approach, the spin vectors defined in `moments` should have the length of the proper magnetic moment on each site, i.e. not necessarily unit vectors.

In [ ]:
# Set up a single unit cell with one atom
a = 1.0
cell = np.array([[-a/2, a/2, a/2],[a/2, -a/2, a/2],[a/2, a/2, -a/2]])

positions = np.array([[0.0 , 0.0, 0.0]])
natom = positions.shape[0]
species = np.ones(natom, dtype=int)

moments = np.zeros((natom, 3))
moments[:, 2] = 1.0

system = SpinSystem(cell, positions, species, moments)


## Step 2 — Define exchange interactions

The `ExchangeShellTable` encodes Heisenberg interactions shell-by-shell.

In the next code cell:
- `J_ij` stores exchange strengths for successive shells
- `r_ij` stores representative bond vectors for each shell
- `add_bond(...)` maps each shell to species-pair interactions

This explicit setup is practical for parameter studies (e.g., truncation tests, sign changes, or shell reweighting).

### Note:
This particular example provides symmetry-reduced exchange interactions. Thus the flag `sym 1` needs to be provided to the input parameters.  
(`sym 1` means that the neighbour vectors will exploit cubic symmetry. Check the manual for further documentation on this flag.)

In [ ]:
exchange = ExchangeShellTable()

# Exchange couplings J_ij in mRy (shell-resolved)
J_ij = [1.33767,  0.7570, -0.05975, -0.0882]
r_ij = np.array([[a/2, a/2, a/2], [0, 0, a], [a, a, a/2], [3*a/2, a/2, a/2]])

for ij, jij in enumerate(J_ij):
    exchange.add_bond(1, 1, ij+1, r_ij[ij], jij)


## Step 3 — Configure input blocks

`ASDInput` organizes UppASD keywords by conceptual blocks (`system`, `initial`, `simulation`, `measurables`).

We use an LLG initial phase (`ip_mode='S'`) with scalar syntax:
- `ip_temp`: initial-phase temperature
- `ip_nstep`: number of initial-phase steps
- `ip_timestep`, `ip_damping`: integration controls

Then we set production simulation controls and request output needed for time-series analysis (`averages` and `totenergy`).

### Note: 
In this particular example we set the initial phase parameters to not thermalize the system properly so we can track the relaxation procedure during the measurement phase. For production simulations, the thermalization needs to be properly done.

In [ ]:
# Define simulation and measurement parameters
inp = ASDInput()
Nx = 10
Ny = 10
Nz = 10

# Define a temperature to ensure same value is given in initial and measurement phases (optional)
Temperature = 30

inp.block("system").set(
    ncell=(Nx, Ny, Nz),
    bc=("P", "P", "P"),
    do_prnstruct=2,
    sym=1,
 )

# Initial phase (LLG) using scalar syntax
inp.block("initial").set(
    ip_mode="S",
    ip_temp=Temperature,
    ip_nstep=100,
    ip_damping=0.1,
    ip_timestep=1.0e-16,
    initmag=1,
 )

# Alternative MC/HB initial phase syntax example:
# inp.block("initial").set(ip_mode="H", ip_temp=Temperature, ip_mcnstep=1000, initmag=1)

# Main simulation settings
inp.block("simulation").set(
    mode="S",
    nstep=5000,
    timestep=1e-15,
    damping=0.001,
    temperature=Temperature,
 )

# Request energy/averages/cumulant output
inp.block("measurables").set(plotenergy=1, do_avrg="Y", do_cumu="Y")

## Step 4 — Prepare workspace and run UppASD

The next two code cells do the full execution workflow:
1. Create a clean run directory and write all required input/interaction files
2. Initialize UppASD, run initial phase, perform measurement, and finalize

Keeping these steps explicit is useful for debugging and for teaching what happens under the hood.

### Note:
The `sim.initialize()` and `sim.finalize()` steps are always needed for any simulation.

In [ ]:
workdir = "bccFe_run"
ws = ASDWorkspace(workdir, clean=True)
ws.prepare(system=system, inp=inp, exchange=exchange)

In [ ]:
sim = UppASDSimulator(ws)
sim.initialize()
sim.run_init_phase()
sim.measure()
sim.finalize()

## Step 5 — Build time-series observables

This single-run tutorial emphasizes **trajectory diagnostics**.

We now parse UppASD outputs and assemble a table with:
- magnetization components and magnitude (`mx`, `my`, `mz`, `m`)
- magnetization spread (`m_std`)
- total energy (`energy_tot`)

This makes relaxation dynamics and equilibration trends visible.


### Note:
The post-processing relies on the `ASDResults` class, which scans the run directory for existing outputs. The list of available outputs is accessed by `ASDResults.available`.

In [ ]:
# Post-processing for a single run: magnetization and energy vs time
results = ASDResults(workdir)
print("Available tables:", results.available)

# Example of a valididy check
if results.averages is None:
    raise ValueError("No averages table found. Ensure do_avrg='Y' in measurables.")
if results.totenergy is None:
    raise ValueError("No totenergy table found. Ensure plotenergy=1 in measurables.")

# Use simulation timestep (seconds) to convert iteration -> physical time
# Not needed if `real_time_measure Y` is used.
sim_timestep = inp.block("simulation").as_dict().get("timestep", np.nan)

# Build compact time-series table from UppASD outputs
ts_df = pd.DataFrame({
    "iter": results.averages["iter"],
    "time_s": results.averages["iter"] * float(sim_timestep),
    "mx_uB": results.averages["mx"],
    "my_uB": results.averages["my"],
    "mz_uB": results.averages["mz"],
    "m_uB": results.averages["m"],
    "energy_tot_mRy": results.totenergy["tot"],
}).sort_values("iter").reset_index(drop=True)

# Print parts of the results to screeen.
display(ts_df.head())
display(ts_df.tail())

## Step 6 — Plot and interpret the trajectories

How to read the upcoming plots:
- A stabilizing `|M|` and `Mz` suggests approach to a steady magnetic state
- A flattening total-energy curve indicates energetic relaxation
- Large late-time fluctuations can imply insufficient averaging or finite-size effects

For careful studies, repeat with larger `nstep`, varied seeds, and multiple temperatures.

Also, vary the protocol for the initial phase to see what is needed for the measured observables to be equilibrated from the start of the measurement phase.

In [ ]:
# Plot average magnetization (μ_B) and total energy (mRy) vs time (seconds)
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)

# Magnetization curve
axes[0].plot(ts_df["time_s"], ts_df["m_uB"], label="|M|", lw=2)
axes[0].plot(ts_df["time_s"], ts_df["mz_uB"], label="Mz", lw=1.5, alpha=0.8)
axes[0].set_title("Average Magnetization vs Time")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Magnetization [μ_B]")
axes[0].legend()
axes[0].grid(alpha=0.3)

# Energy curve
axes[1].plot(ts_df["time_s"], ts_df["energy_tot_mRy"], color="tab:red", lw=2)
axes[1].set_title("Total Energy vs Time")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Total Energy [mRy]")
axes[1].grid(alpha=0.3)

plt.show()